#### Prepare the data

In [2]:
import csv 
import cv2 
import numpy as np

path = 'data/IMG/'
images = [] 
measurements = [] 
with open('data/driving_log.csv') as csvfile: 
    reader = csv.reader(csvfile)
    for line in reader: 
        filename = line[0].split('/')[-1]
        steering = line[3]
        images.append(cv2.imread(path + filename))
        measurements.append(float(steering))
        
X = np.array(images)
y = np.array(measurements)

In [3]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

batch_size = 64 
epochs = 10 

img_rows, img_cols, img_chls = X[1].shape

In [6]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=X.shape[1:]))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1))

model.compile(loss='mse',
              optimizer='adam'
             )

model.fit(X, y,
          batch_size=batch_size,
          validation_split = 0.2,
          epochs=epochs,
          verbose=1, 
          shuffle=True
         )

Train on 1472 samples, validate on 368 samples
Epoch 1/10
 128/1472 [=>............................] - ETA: 601s - loss: 1730514255.3340

KeyboardInterrupt: 